In [2]:
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [4]:
X,y=datasets.load_iris(True)

#将目标值为2的样本去除
cond=y!=2
X=X[cond]
y=y[cond]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [6]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
proba_=lr.predict_proba(X_test)

In [7]:
w=lr.coef_
b=lr.intercept_
#为了便于点乘，先看看参数的形状

print(X_test.shape)
print(w.shape,b.shape)

(20, 4)
(1, 4) (1,)


In [8]:
#计算线性结果
z=X_test.dot(w.T)+b

#带入sigmoid函数计算概率,p是类别为1的概率
p=1/(1+np.e**(-z))

#将类别为0的概率和类别为1的概率进行级联
prob_1=np.c_[1-p,p]

In [ ]:
def train_report(data,p_cols,param_grid = {'C':[100,10,1.1,1.05,1,.01,.015,0.017,.02],'penalty':['l2'], 
                                 'class_weight':[None,'balanced']}):
    
    ## split
#     X_train, X_test, y_train, y_test = train_test_split(
#              X,y, test_size=0.33, random_state=42)
    X_train, X_test = data.query("train_or_test=='train'")[p_cols],data.query("train_or_test=='test'")[p_cols]
    y_train, y_test = data.query("train_or_test=='train'")['label'],data.query("train_or_test=='test'")['label']
    ## train
    lr = LogisticRegression(solver='liblinear',random_state = 42)
    clf = GridSearchCV(lr,
                   param_grid = param_grid,
                   n_jobs=-1,scoring='f1',cv=4)
    clf.fit(X_train,y_train)
    
    ## summary
    print(f'test: {f1_score(y_test,clf.predict(X_test))}')
    print(f'train: {f1_score(y_train,clf.predict(X_train))}')
    print(f'total: {f1_score(data.label,clf.predict(data[p_cols]))}')   
    print('best_params: ',f'{clf.best_params_}')
    
    fi = pd.DataFrame({'feature':X_train.columns,'imp':clf.best_estimator_.coef_[0]})\
            .sort_values('imp',ascending=False)\
            .eval('imp_percent = imp/imp.sum()')\
            .set_index('feature')
    return fi